In [1]:
import pandas as pd

train = pd.read_csv('train.txt', sep=';', header=None, names=['text', 'label'])
test = pd.read_csv('test.txt', sep=';', header=None, names=['text', 'label'])
val = pd.read_csv('val.txt', sep=';', header=None, names=['text', 'label'])

train.head()

,text,label
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [2]:
# encode label in the dataset
from sklearn.preprocessing import LabelEncoder

# Create LabelEncoder object
le = LabelEncoder()
# Fit the encoder to the pandas column
le.fit(train['label'])
# View encoder mapping
dict(zip(le.classes_,le.transform(le.classes_)))

{'anger': 0, 'fear': 1, 'joy': 2, 'love': 3, 'sadness': 4, 'surprise': 5}

In [3]:
# transform the labels in train, test and val

train['label']=le.transform(train['label'])
test['label'] = le.transform(test['label'])
val['label'] = le.transform(val['label'])

train.head()

,text,label
0,i didnt feel humiliated,4
1,i can go from feeling so hopeless to so damned...,4
2,im grabbing a minute to post i feel greedy wrong,0
3,i am ever feeling nostalgic about the fireplac...,3
4,i am feeling grouchy,0


In [4]:
import sentence_transformers

# Load the sentence transformer model
encoder = sentence_transformers.SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

# Vectorized code
train['embeddings'] = train['text'].apply(lambda sentence: encoder.encode(sentence))
test['embeddings'] = test['text'].apply(lambda sentence: encoder.encode(sentence))
val['embeddings'] = val['text'].apply(lambda sentence: encoder.encode(sentence))

In [5]:
train['embeddings'].shape

(16000,)

In [6]:
train['embeddings'][0].shape

(768,)

In [7]:
import tensorflow as tf
import numpy as np

train_embeddings = np.stack(train['embeddings'].values)
val_embeddings = np.stack(val['embeddings'].values)

train_labels = tf.keras.utils.to_categorical(train['label'], num_classes=6)
val_labels = tf.keras.utils.to_categorical(val['label'], num_classes=6)

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(1024, input_shape=(768, ), activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(6, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_embeddings, train_labels, epochs=40, batch_size=1024, validation_data=(val_embeddings, val_labels))

Epoch 1/40
16/16 [==============================] - 1s 36ms/step - loss: 1.2654 - accuracy: 0.5462 - val_loss: 1.0219 - val_accuracy: 0.6230
Epoch 2/40
16/16 [==============================] - 0s 9ms/step - loss: 0.9778 - accuracy: 0.6409 - val_loss: 0.9428 - val_accuracy: 0.6410
Epoch 3/40
16/16 [==============================] - 0s 9ms/step - loss: 0.8960 - accuracy: 0.6690 - val_loss: 0.8920 - val_accuracy: 0.6625
Epoch 4/40
16/16 [==============================] - 0s 9ms/step - loss: 0.8391 - accuracy: 0.6886 - val_loss: 0.8741 - val_accuracy: 0.6670
Epoch 5/40
16/16 [==============================] - 0s 10ms/step - loss: 0.7873 - accuracy: 0.7094 - val_loss: 0.8445 - val_accuracy: 0.6870
Epoch 6/40
16/16 [==============================] - 0s 11ms/step - loss: 0.7500 - accuracy: 0.7233 - val_loss: 0.8193 - val_accuracy: 0.6865
Epoch 7/40
16/16 [==============================] - 0s 9ms/step - loss: 0.7089 - accuracy: 0.7368 - val_loss: 0.8182 - val_accuracy: 0.6870
Epoch 8/40
16/16 

In [8]:
# test model 

test_embeddings = np.stack(test['embeddings'].values)

# predict
preds = model.predict(test_embeddings)

# get the index of the max probability
preds = np.argmax(preds, axis=1)

# get the accuracy
from sklearn.metrics import accuracy_score
acc = accuracy_score(test['label'], preds)

63/63 [==============================] - 0s 3ms/step


In [9]:
print(acc)

0.726


In [13]:
import sentence_transformers
import tensorflow as tf
import numpy as np
# Load the sentence transformer model
encoder = sentence_transformers.SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
model = tf.saved_model.load("model")
def emotion_detect(text):
    x = encoder.encode(text)
    x = np.array(x).reshape(1, 768)
    pred = model(x)
    pred = np.argmax(pred)
    dict = {'anger': 0, 'fear': 1, 'joy': 2, 'love': 3, 'sadness': 4, 'surprise': 5}
    return list(dict.keys())[list(dict.values()).index(pred)]

emotion_detect("hey there how are you?")

'joy'

In [17]:
emotion_detect("fuck you")

'anger'

In [12]:
# save the model to disk
tf.saved_model.save(model, "model")

INFO:tensorflow:Assets written to: model\assets
